# Sandbox

In [1]:
import sys
sys.path.append('../sddip')

In [2]:
#%run ../scripts/create_result_directories.py

In [3]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp

import sddip.tree as tree
import sddip.utils as utils
import sddip.config as config


## Data Processing

In [4]:
test_case_raw_dir = "WB2/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")

scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\t")
branch_df = pd.read_csv(branch_file_raw, delimiter="\t")
gen_df = pd.read_csv(gen_file_raw, delimiter="\t")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\t")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\t")

### Bus Data

In [5]:
bus_df

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1,3,0,0,0,0,1,0.964,0,0,1,1.05,0.95
1,2,1,350,-350,0,0,1,1.000,-65,0,1,1.05,0.95


In [6]:
bus_df.dtypes

bus_i       int64
type        int64
Pd          int64
Qd          int64
Gs          int64
Bs          int64
area        int64
Vm        float64
Va          int64
baseKV      int64
zone        int64
Vmax      float64
Vmin      float64
dtype: object

### Branch Data

In [7]:
branch_df

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
0,1,2,0.04,0.2,0,990000,0,0,0,0,1,-360,360


### Generator Data

In [8]:
gen_df

,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
0,1,400,100,400,-400,0.964,100,1,600,0,...,0,0,0,0,0,0,0,0,0,0


### Generator Cost Data

In [9]:
gen_cost_df

,type,startup,shutdown,n,c2,c1,c0
0,2,0,0,3,0,2,0


In [10]:
scenario_df

,t,n,p,Pd
0,1,1,1.0,350
1,2,1,0.5,300
2,2,2,0.5,200


### Power Transfer Distribution Factor

In [11]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

## Recombining Tree

In [12]:
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()

node_params_df = scenario_df.drop(["t", "n"], axis=1)
node_params_dicts = node_params_df.to_dict("records")

s_tree = tree.RecombiningTree(n_nodes_per_stage)

s_tree.params["gc"] = gen_cost_df.c1
s_tree.params["suc"] = gen_cost_df.startup
s_tree.params["sdc"] = gen_cost_df.startup
s_tree.params["pg_min"] = gen_df.Pmin
s_tree.params["pg_max"] = gen_df.Pmax
s_tree.params["pl_max"] = branch_df.rateA
s_tree.params = {k: v.values.tolist() for k,v in s_tree.params.items()}
s_tree.params["ptdf"] = ptdf

for stage in s_tree:
    for node in stage:
        params = node_params_dicts.pop(0)
        node.prob = params.pop("p")
        node.params = params